In [9]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler

In [10]:
%%time

# ładowanie zbioru danych
iris = load_iris()

X, y, iris_classes = iris.data, iris.target, iris.target_names

# dzielimy dane na treningowe i testowe w stosunku 0.7/0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# skalowanie danych treningowych i testowych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

CPU times: total: 0 ns
Wall time: 3.99 ms


In [11]:
%%time

# uzycie pojedynczego modelu
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier()

clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print("Pojedynczy wynik: ", score)

Pojedynczy wynik:  0.9333333333333333
CPU times: total: 0 ns
Wall time: 3.99 ms


In [12]:
from sklearn.model_selection import KFold

# stworzenie KFold obiektu z 5 foldami i tasowaniem danych
kf = KFold(n_splits=5, shuffle=True, random_state=1)

models = []
scores = []

# iteracja przez poszczególne foldy
for fold_nr, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train = X[train_idx]
    X_test =X[test_idx]

    y_train = y[train_idx]
    y_test = y[test_idx]

    # skalowanie danych wewnątrz poszczeŋólnych iteracji
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
  
    # stworzenie klasyfikatora dla poszczególnej iteracji
    # wytrenowanie klasyfikatora na danych treningowych
    clf = SGDClassifier(random_state=1).fit(X_train, y_train)

    # dodawanie klasyfikatora do tablicy klasyfikatorów
    models.append(clf)
    # dodawanie wyniku dla danych testowych do tablicy
    scores.append(clf.score(X_test, y_test))

print("wyniki poszczególnych foldów: ", scores)
print("sredni wynik wszystkich foldów: ", np.array(scores).mean())

wyniki poszczególnych foldów:  [0.8666666666666667, 0.9333333333333333, 0.9666666666666667, 0.9, 0.9333333333333333]
sredni wynik wszystkich foldów:  0.9199999999999999


In [13]:
from sklearn.model_selection import cross_val_score, cross_val_predict

# stworzenie klasyfikatora
clf = SGDClassifier(random_state=1)

# uzycie metory cross_val_Score do sprawdzenia działania naszego modelu na róznych podziałach
cv_score = cross_val_score(clf, X, y, cv=5)
print("wynik kroswalidacji: ", cv_score)
print("średni wynik wszystkich foldów: ", cv_score.mean())

wynik kroswalidacji:  [0.83333333 0.93333333 0.8        0.7        0.9       ]
średni wynik wszystkich foldów:  0.8333333333333334


In [14]:
# przykład podziału który będzie złą estymatą dla działania naszego klasyfikatora

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99, 0.01], flip_y=0, random_state=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=3)
# summarize
train_0, train_1 = len(y_train[y_train==0]), len(y_train[y_train==1])
test_0, test_1 = len(y_test[y_test==0]), len(y_test[y_test==1])
print('>Train: 0=%d, 1=%d, Test: 0=%d, 1=%d' % (train_0, train_1, test_0, test_1))

>Train: 0=790, 1=10, Test: 0=200, 1=0


In [15]:
#Przykład podziału za pomocą stratyfikacji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=3, stratify=y)
# summarize
train_0, train_1 = len(y_train[y_train==0]), len(y_train[y_train==1])
test_0, test_1 = len(y_test[y_test==0]), len(y_test[y_test==1])
print('>Train: 0=%d, 1=%d, Test: 0=%d, 1=%d' % (train_0, train_1, test_0, test_1))

>Train: 0=792, 1=8, Test: 0=198, 1=2


In [16]:
# wcześniejszy przykład z użyciem stratifiedKFold

from sklearn.model_selection import StratifiedKFold


# stworzenie KFold obiektu z 5 foldami i tasowaniem danych
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

models = []
scores = []

# iteracja przez poszczególne foldy
for fold_nr, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train = X[train_idx]
    X_test =X[test_idx]

    y_train = y[train_idx]
    y_test = y[test_idx]

    # skalowanie danych wewnątrz poszczeŋólnych iteracji
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
  
    # stworzenie klasyfikatora dla poszczególnej iteracji
    # wytrenowanie klasyfikatora na danych treningowych
    clf = SGDClassifier(random_state=1).fit(X_train, y_train)

    # dodawanie klasyfikatora do tablicy klasyfikatorów
    models.append(clf)
    # dodawanie wyniku dla danych testowych do tablicy
    scores.append(clf.score(X_test, y_test))


print("wyniki poszczególnych foldów: ", scores)
print("średni wynik wszystkich foldów: ", np.array(scores).mean())

wyniki poszczególnych foldów:  [0.995, 0.985, 0.975, 0.965, 0.99]
średni wynik wszystkich foldów:  0.982
